In [1]:
import io
import icu
import json

with io.open("rawdata/kangxi-radicals-compat.json", "r", encoding = "utf-8") as r:
    radicals = json.loads(r.read())
    r.close()

# convert kRSUnicode or kRSKangXi into wiki sortkey
def rscode_to_wikikey(rscode):
    tokens = rscode.split(".")
    if simp := tokens[0].endswith("'"):
        tokens[0] = tokens[0][:-1]
    key = radicals["alternative"][int(tokens[0]) - 1] if simp else radicals["original"][int(tokens[0]) - 1]
    key += "{:+03d}".format(int(tokens[1])).replace("+", "")
    return key

with io.open("data/Unihan_IRGSources.json", "r", encoding = "utf-8") as u:
    dbObj = json.loads(u.read())
    u.close()

newObj = {} # sortkeys
newObj2 = {} # reverse sortkeys
specialObj = {} # multiple sortkeys
for hanChar in dbObj:
    values = dbObj[hanChar]["kRSUnicode"]
    values[:] = [rscode_to_wikikey(value) for value in values]
    newObj[hanChar] = values
    if len(values) > 1:
        specialObj[hanChar] = values
    for value in values:
        if value not in newObj2:
            newObj2[value] = []
        newObj2[value].append(hanChar)

with io.open("data/sortkey.json", "w", encoding = "utf-8") as j:
    json.dump(newObj, j, ensure_ascii = False, separators = (",", ":"))
    j.close()
with io.open("data/sortkey-rev.json", "w", encoding = "utf-8") as k:
    json.dump(newObj2, k, ensure_ascii = False, separators = (",", ":"))
    k.close()
with io.open("data/sortkey-multiple.json", "w", encoding = "utf-8") as l:
    json.dump(specialObj, l, ensure_ascii = False, separators = (",", ":"))
    l.close()